In [2]:
from IPython.display import Image

## Problème 1

En supposant que le corps est fait d'eau pure et suite à la consultation des données XCOM du NIST, nous avon obtenus les coefficients d'atténuation suivant
\begin{align}
\mu_c = 0,1929\ \text{cm$^2$/g}, \ \ \ \ \mu_{n-c} = 0,1824\ \text{cm$^2$/g}.
\end{align}
Le premier indique l'atténuation de l'eau à 70 keV dans un contexte de diffusion chérente tandis que le deuxième est lors d'une diffusion non cohérente. On metionne dans [1] que la diffusion est cohérente lorsque l'énergi du rayonnement incident sur la cible est sous l'énergie d'ionisation de l'atome. Ainsi, dans le cas de l'eau, ayant énergie d'ionisation atomique de près de 13 eV [2], un rayonnement de 70 keV résulte en une diffusion non cohérente. 

L'atténuation de l'intensinté d'un faisceau est donnée par 
\begin{align}
\frac{I}{I_0} = exp(-\int \mu(x) dx).
\end{align}
Or, dans le cas de l'eau, l'atténuation est constante sur le trajet optique. Ainsi, on peut calculer la portion de rayonnement résultante de l'atténuation comme suit
\begin{align}
\frac{I}{I_0} = e^{-\mu l}
\end{align}

Il est à noter que la masse volumique de l'eau étant de 1 g/cm$^3$, nous avons les même grandeurs d'atténuation que ci-haut pour des unités de cm$^{-1}$. 

Bref, 
\begin{align}
 \frac{I}{I_0}\bigg|_{l=5cm} = 0.402, \ \ \ \ \frac{I}{I_0}\bigg|_{l=20cm} = 0.0260
\end{align}

## Reconstruction par rétroprojection

Nous devons faire la reconstruction d'une tomographie à partir d'un sinogramme. Le sinogramme fourni comprend l'atténuation en fonction de la position sur le détecteur en fonction de l'angle de la projection. \\

Nous allons ainsi utiliser une méthode connue sous le nom de voxel-driven, où on passe voxel par voxel calculer les contributions des différentes projections.\\

### Problème 2
Créez un laminogramme des données de projection fournies, et comparez au fantôme numérique après avoir identifié le code. Cette image serait-elle acceptable pour un diagnostic médical?\\

#### Algorithme de reconstruction
Pour créer le laminogramme, nous utilisons une méthode maison on trois étapes. Premièrement, on calcule la distance entre l'axe principal du détecteur et le voxel à l'aide d'une transformation de rotation.
\begin{align}
d_{axe} = (x0-x)\cdot\cos(theta) - (y0-y)\cdot\sin(\theta)
\end{align}
où x et y représentent les indices de la matrice de voxels, et $x_0, y_0$ sont simplement le point de rotation du système, qu'on pose comme étant le centre de la matrice de voxels. De façon plus imagée, on calcule $d_{axe}$ de la figure suivante.\\
Ajouter figure?\\
Ensuite, on ajoute un facteur de dilatation sur nos distances, ce qui équivaut à une paramétrisation de nos projections. Cela fait en sorte que la projection de notre matrice de voxels va toujours couvrir tout notre espace peu importe l'angle. Le facteur que nous utilisons est de 
\begin{align}
\frac{\sqrt{2}\cdot len(sinogramme)}{2\cdot len(voxels)}
\end{align}
Qui équivaut à la multiplication du facteur de conversion entre les deux tailles et la facteur $\sqrt{2}$, qui s'assure que notre détecteur couvre aussi la matrice en entier lorsqu'elle est à 45$^{\circ}$\\

Finalement, on convertis les distances calculées en indices, en les arrondissant à l'entier le plus proche, après avoir additionnant la moitié de la longueur de la projection. Cette addition vient du fait qu'on considère la distance comme originant de la droite qui coupe les projections en leur milieu, et sont positives et négatives. Il est important de noter que toutes ces étapes peuvent se faire sans l'usage de boucles \textit{for}, et qu'implémenter le \textit{slicing} ici peut augmenter la vitesse d'exécution d'une facteur 100 sans changer la solution.\\

Il en résulte un algorithme qui solutionne 720~projections de 336~pixels dans un laminogramme de 96~voxels dans des temps inférieurs à une seconde.\\



<img style="float: center;" src="phantom.png" >
<img style="float: center;" src="Backproject_non_filtre.png" >

### Comparaison fantôme-reconstruction
Lorsqu'on compare le fantôme à la reconstruction, on remarque instantanément les erreurs approtées par la reconstruction. Ce qui est flagrant est que, près des zones de fort contraste, il y a présence d'un fort \textit{flou}, comme si la zone de haute atténuation débordait de la frontière. Par contre, on voit très bien les contours des grandes structures de l'image, tel que les deux lobes (poumons?) foncés. Finalement, les zones blanches (os?) sont visibles mais pas claires, et les petits points dans les poumons sont complétement invisibles. Sommairement, il est très difficile de distinguer les détails de l'image, et elle ne peut pas être utilisée pour effectuer un diagnostique médical.

## Rétroprojection filtrée
Comme le laminogramme n'est pas une solution exacte de la reconstruction TDM, il est nécessaire de filtrer les basses fréquences du sinogramme avant de le reconstruire pour obtenir une meilleure reconstruction.\\

### Problème 3
Implémentez le filtre passe-haut f(u)  = |u| sur chaque ligne du sinogramme. Quelles sont les différences entre le sinogramme d'origine et le sinogramme filtré?\\

#### Implémentation
L'implémentation du filtre est très simple lorsqu'on suit la documentation de \tb{numpy.fft}. Il suffit de faire la fft de chaque ligne, puis d'en trouver la \textit{fftfreq} et d'exécuter \textit{fftshift} sur les deux. On peut alors effectuer l'opération valeur absolue, en faisant attention de prenre le conjugué de cette valeur, puisqu'elle inverse la partie imaginaire de notre fft. Finalement, on fait toutes les transformations inverses et on enregistre le sinogramme filtré.\\



<img style="float: center;" src="Non-filtre.png" >
<img style="float: center;" src="filtre.png" >

#### Comparaison
On voit que la forme générale du sinogramme reste inchangé après la filtration, mais que, comme on s'y attendait, les zones de basses fréquences ont une \textit{baseline} qui décroît plus on séloigne des contours. Il s'agit donc du problème inverse qu'on avait lors de la reconstruction, et on imagine que les deux vont se contrebalancer lors de la reconstruction. Ce qui explique la baisse de la \textit{baseline} est le simple fait que les zones de valeurs continues sont représentées par des basses fréquences dans l'espace de Fourier. Comme le filtre que nous avons appliqué atténue les basses fréquences, il est tout à fait logique que les grandes zones uniformes de notre sinogramme aient une amplitude qui tende vers zéro.

### Problème 4
Effectuer une rétropropagation filtrée du sinogramme filtré, et comparer l'image obtenue avec le fantôme. Quelles sont les différences? Où sont elles et quelle est leur fréquences?\\

#### Reconstruction
On entre simpement le sinograme filtré dans notre algorithme du numéro précédent et on obtient une reconstruction bien plus près du fantôme qu'initialement.\\



<img style="float: center;" src="Backproject_fft_filtre.png" >

#### Comparaison
On voit tout de suite l'amélioration introduite par la filtration du sinogramme. En effet, il est maintenant possible d'identifier facilement les os et les parois internes des poumons. On peut même maintenant voir les os plus minces de la colonne vertébrale du patient, une nette amélioration! Par contre, il y a toujours présence de bruit dans les zones de faible constraste. Effectivement, si on regarde simplement les zones noires entourant le patient, on voit tout de suites un genre de bruit de hautre fréquence. Ce bruit est présent un peu partout dans l'image, et est plus intense au centre des grandes zones uniformes. Ceci est dérangeant si l'on essaie d'identifier les structures dans les poumons du patient visibles dans le fantôme. En effet, dans la reconstruction, il est facile de les méprendre pour du bruit, et certaines ne sont simplement pas visibles.

### Problème 5
On tente d'améliorer davantage en faisant l'interpolation linéaire des des pixels adjacents du détecteur pour obtenir une valeur plus précise d'atténuation. Comment l'image s'est-elle améliorée?

#### Implémentation
Comme on ne veut pas nuire au temps d'exécution de notre algorithme, il faut implémenter l'interpolation linéaire tout en faisant du \textit{slicing}. Pour ce faire, on prend les indices supérieurs (sup) et inférieurs (inf) en remplaçant notre arrodis par un \tb{np.ceil()} et un \tb{np.floor()}, qui arrondissent vers le haut et le bas respectivement, peu importe le reste. On prend aussi en note ce reste avec l'opération \tb{$dist\% 1$} avec "$\%$" représentant l'opération modulo et $dist$ notre $d_{axe}$. Finalement, on utilise ces résultats pour faire notre interpolation, qui se traduit par la ligne de code:





In [ ]:
image[j,i] += np.sum(reste*(sup-inf) + inf)

où, si on pense à une fonction linéaire $ax + b$, \tb{reste} resprésente $x$, \tb{(sup-inf)} représente $a$ et \tb{inf} représente $b$. Il en résulte alors une interpolation linéaire entre les deux pixels.\\

<img style="float: center;" src="Laminogram.png" >

#### Comparaison
On voit instantanément une nette amélioration de la qualité de l'image. En effet, les artéfacts de haute fréquences semblent avoir complétement disparus, et laissent place à de faibles erreurs continues suivant les sections de haut contraste. Bien que ces erreurs semblent quasiment négligeables, les petites structures dans les poumons ne sont pas parfaitement représentées si l'on compare la reconstruction au fantôme fourni. Par exemple, il est possible de les détecter et de les positionner, mais on ne peut pas calculer leur volume de façon rigoureuse avec cette méthode.\\

Aussi, les calculs supplémentaires doublent presque le temps de calcul, puisqu'on rajoute une quelques itération pour chaque voxel. Si on moyenne le temps d'exécution sur 20 exécutions du programme (sans \tb{print} ni \tb{savefile}), on obtient un temps de $0.81$~s pour l'exécution de l'algorithme normal, et de $1.39$~s pour l'exécution avec interpolation linéaire. Nous avons donc un grand gain en qualité d'image pour un facteur de décélération de $1.73$.

## Problème 6

Décrire la méthode utilisée pour la reconstruction par TdF et mettre le code

## Problème 7

Suite à l'implémentation de la fonction de reconstruction par tranches de Fourier présentée au numéro 6, nous obtenons le résultat suivant pour une taille de voxel de 0.4 et un nombre de 96.

<img style="float: center;" src="Low-Res.png" >

On distingue donc relativement bien les différentes parties du torse. Par contre, on remarque la présence d'artéfacts de reconstruction qui proviennent de l'échantillonnage. Ceux-ci seront discuté au numéro 9. On voit aussi qu'une partie de l'image est répétée dans les côtés droit et gauche de la figure. Ce résultat s'explique par la périodicité de l'espace de Fourier. Bref, nous en convenons que l'image obtenue par la méthode présentée au numéro 6 donne un résultat convaincant, i.e. on a bel et bien un torse, mais celle-ci ne pourrait pas être utilisée pour faire un diagnostic adéquat étant donné la présence des artéfacts de reconstruction et de la faible résolution.

## Problème 8

En utilisant la méthode présentée au numéro 6, mais cette fois avec une dimension des voxels de 0.2 et un nombre de 192, on obtient l'image suivante

<img style="float: center;" src="Hi-Res.png" >

Tout d'abord, on peut remarquer qu'une augmentation du nombre de voxel mène à un meilleur contraste pour les petites structures, mais que des artéfacts de reconstruction sont toujours présents. Toutefois, étant donné la meilleure résolution, un diagnostic effectué à partir de cette image serait mieux capable de cerner la position des objets ou des tumeurs, le cas échéant. Il semblerait donc légitime de dire que cette image pourrait être utilisée lors d'un diagnostic, en sachant toutefois que les zones plus blanches sur les côtés de la reconstruction ne représentent pas une grosse tumeur.  

## Problème 9

Suite à l'implémentation de différentes méthodes de reconstruction tomographiques, il semble naturel de vouloir comparer leur temps d'exécution. Pour ce faire, nous avons exécuté 20 fois chacune ces méthodes, en avons pris la moyenne et avons associé une incertitude correspondant à 2 écarts types de la distribution de temps d'exécution obtenus. Nous avons donc que la méthode la plus lente est la méthode des tranches de Fourier à haute résolution (i.e. 192 voxels ayant un taille de 0.2) avec un temps moyen de calcul de 2.47±0.01 secondes, s'en suit la méthode de rétroprojection avec un temps de calcul moyen de 1.18±0.01 secondes. Bref, la méthode la plus efficace s'avère être celle des tranches de Fourier à basse résolution (i.e. 96 voxels ayant un taille de 0.4) avec un temps d'exécution de 0.92±0.02 secondes. En ce qui concerne les boucles for imbriquées lors de ces méthode, chacune d'entre elle en possède 2, soit une pour les lignes et l'autre pour les colonnes. Un gestion adéquate des angles par slicing est utilisée pour la méthode de rétroprojection tandis qu'un seul angle est considéré pour chaque point de la reconstruction pour la reconstruction par tranches de Fourier. De plus, la méthode de correspondance sinogram-reconstruction n'est pas la même pour la rétroprojection et pour les tranches de Fourier, d'où l'augmentation du temps de calcul.

## Problème 10

Description de la méthode d'interpolation implémentée par Charles.



## Références
[1] Radiopedia, S. Price et al. https://radiopaedia.org/articles/coherent-scattering 

[2] WebBook de Chimie du NIST https://webbook.nist.gov/cgi/cbook.cgi?ID=C7732185&Mask=20